In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import bz2
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
import re
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
%matplotlib inline

In [2]:
def get_labels_and_texts(file):
    labels = []
    texts = []
    for line in bz2.BZ2File(file):
        x = line.decode("utf-8")
        labels.append(int(x[9]) - 1)
        texts.append(x[10:].strip())
    return np.array(labels), texts
train_labels, train_texts = get_labels_and_texts('train.ft.txt.bz2')
test_labels, test_texts = get_labels_and_texts('test.ft.txt.bz2')

In [3]:
train_labels[0]

1

In [4]:
train_texts[0]

'Stuning even for the non-gamer: This sound track was beautiful! It paints the senery in your mind so well I would recomend it even to people who hate vid. game music! I have played the game Chrono Cross but out of all of the games I have ever played it has the best music! It backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. It would impress anyone who cares to listen! ^_^'

In [12]:
train_labels

array([1, 1, 1, ..., 0, 0, 1])

In [29]:
train_labels=train_labels[0:500]

In [30]:
train_texts=train_texts[0:500]

In [15]:
import re
NON_ALPHANUM = re.compile(r'[\W]')
NON_ASCII = re.compile(r'[^a-z0-1\s]')
def normalize_texts(texts):
    normalized_texts = []
    for text in texts:
        lower = text.lower()
        no_punctuation = NON_ALPHANUM.sub(r' ', lower)
        no_non_ascii = NON_ASCII.sub(r'', no_punctuation)
        normalized_texts.append(no_non_ascii)
    return normalized_texts
        
train_texts = normalize_texts(train_texts)
test_texts = normalize_texts(test_texts)

In [16]:
train_texts[0]

'stuning even for the non gamer  this sound track was beautiful  it paints the senery in your mind so well i would recomend it even to people who hate vid  game music  i have played the game chrono cross but out of all of the games i have ever played it has the best music  it backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras  it would impress anyone who cares to listen    '

In [19]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(binary=True)
cv.fit(train_texts)
X = cv.transform(train_texts)
X_test = cv.transform(test_texts)

In [20]:
X_test

<400000x5778 sparse matrix of type '<class 'numpy.int64'>'
	with 18647020 stored elements in Compressed Sparse Row format>

#Building Logistics Regression Model

In [42]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

train_texts, val_texts, train_labels, val_labels = train_test_split(
    X, train_labels, random_state=57643892, test_size=0.2)

for c in[0.01, 0.05, 0.25, 0.5, 1]:
    
    lr = LogisticRegression(C=c)
    lr.fit(train_texts, train_labels)
    print("Accuracy for C=%s: %s"
           % (c, accuracy_score(val_labels, lr.predict(val_texts))))



Accuracy for C=0.01: 0.73
Accuracy for C=0.05: 0.77
Accuracy for C=0.25: 0.78
Accuracy for C=0.5: 0.79
Accuracy for C=1: 0.79


In [41]:
test_labels[28]

1

In [40]:
test_texts[28]

'wish i had gotten this sooner   ive got a lamp in the corner of my room behind my desk thats a complete pain in the arse to turn on and off  ive been using this with the lamp for a month now and it works perfectly  added a little velcro and now i have a light switch where ever i want  under my desk  shelf  etc '

In [76]:
train_texts.shape

(400, 5778)